## This notebook contains e2e analysis used to allocate tolerances for each $\color{red}{\text{High Spatial Frequency}}$ mode for a segmented telescope.

### Import necessary libraries and scripts

In [ ]:
import os
os.chdir("/Users/asahoo/repos/PASTIS")
import time
from shutil import copy
from astropy.io import fits
import astropy.units as u
import hcipy
import numpy as np
import pastis.util as util    
from pastis.config import CONFIG_PASTIS 
from pastis.e2e_simulators.luvoir_imaging import LuvoirA_APLC 
from pastis.e2e_simulators.generic_segmented_telescopes import SegmentedAPLC
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import griddata
import exoscene.image
import exoscene.star
import exoscene.planet
from exoscene.planet import Planet
from astropy.io import fits as pf
from pastis.analytical_pastis.temporal_analysis import req_closedloop_calc_batch
from matplotlib.colors import TwoSlopeNorm
import matplotlib.ticker as ticker

### Add the following to config ini in future

In [ ]:
coronagraph_design = 'small'
nb_seg = CONFIG_PASTIS.getint('LUVOIR', 'nb_subapertures')
nm_aber = CONFIG_PASTIS.getfloat('LUVOIR', 'calibration_aberration') * 1e-9
sampling = CONFIG_PASTIS.getfloat('LUVOIR', 'sampling')

### Define and create directory

In [ ]:
data_dir = "/Users/asahoo/Documents/ultra/hi_spatial"
repo_dir = "/Users/asahoo/repos/PASTIS"
overall_dir = util.create_data_path(data_dir, telescope='luvoir_'+coronagraph_design)
resDir = os.path.join(overall_dir, 'matrix_numerical')

os.makedirs(resDir, exist_ok=True)

### Instantiate LUVOIR-A

In [ ]:
optics_input = os.path.join(util.find_repo_location(), CONFIG_PASTIS.get('LUVOIR', 'optics_path_in_repo'))
luvoir = LuvoirA_APLC(optics_input, coronagraph_design, sampling)

### create high frequency ripple mirror

In [ ]:
max_HI = 10  #this can be imported from the config file.
luvoir.create_ripple_mirror(max_HI)
n_HI = luvoir.ripple_mirror.num_actuators 
print("Number ripple mirror actuators:",n_HI)

### plot the basis vector for ripple mirror

In [ ]:
# fourier_grid = hcipy.make_pupil_grid(dims=max_HI, diameter=max_HI)  
# fourier_basis = hcipy.mode_basis.make_fourier_basis(luvoir.pupil_grid, fourier_grid, sort_by_energy=True)
# ripple_mirror = hcipy.optics.DeformableMirror(fourier_basis)
# m = ripple_mirror.influence_functions[]
# hcipy.imshow_field(m)

###  make a video

In [ ]:
# mw = hcipy.FFMpegWriter('modes.mp4')
# for m in ripple_mirror.influence_functions:
#     plt.clf()
#     hcipy.imshow_field(m)
#     mw.add_frame()
# mw.close()
# mw

### plot the input aberration vector

In [ ]:
HI_modes = np.zeros(n_HI)
HI_modes[60] = (nm_aber)/2
luvoir.ripple_mirror.actuators = HI_modes 
aberrated_coro_psf, ref= luvoir.calc_psf(display_intermediate=True, return_intermediate='efield', norm_one_photon=True)

In [ ]:
#hcipy.imshow_field((ref['ripple_mirror']).phase, mask=luvoir.aperture, cmap='RdBu')

### compute static dark hole contrast $c_{0}$

In [ ]:
HI_modes = np.zeros(n_HI)
luvoir.ripple_mirror.actuators = HI_modes
ideal_coro_psf, ref= luvoir.calc_psf(ref =True, display_intermediate=False, norm_one_photon=True)
norm = np.max(ref)
dh_intensity = (ideal_coro_psf / norm) * luvoir.dh_mask
contrast_floor = np.mean(dh_intensity[np.where(luvoir.dh_mask != 0)])
print(f'norm: {norm}',f'constrast floor: {contrast_floor}')

### compute HI spatial frequency pastis matrix

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
nonaberrated_coro_psf, efield = luvoir.calc_psf(display_intermediate=False, return_intermediate='efield',norm_one_photon=True)
Efield_ref = nonaberrated_coro_psf.electric_field

In [ ]:
print('Generating the E-fields for harris modes in science plane')
print(f'Calibration aberration used: {nm_aber} m')

start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

for i in range(0, n_HI):
    print(f'Working on HI spatial frequency, basis: {i+1}')
    
    # Apply calibration aberration to used mode
    HI_modes = np.zeros(n_HI)
    HI_modes[i] = (nm_aber)/2 
    luvoir.ripple_mirror.actuators  = HI_modes
   
    # Calculate coronagraphic E-field and add to lists
    aberrated_coro_psf, inter = luvoir.calc_psf(display_intermediate=False, return_intermediate='efield',norm_one_photon=True)
    focus_field1 = aberrated_coro_psf
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

In [ ]:
mat_HI = np.zeros([n_HI, n_HI])
for i in range(0, n_HI):
    for j in range(0, n_HI):
        test = np.absolute((focus_fieldS[i].electric_field - Efield_ref) * np.conj(focus_fieldS[j].electric_field - Efield_ref))
        dh_test = (test / norm) * luvoir.dh_mask
        contrast = np.mean(dh_test[np.where(luvoir.dh_mask != 0)])
        mat_HI[i, j] = contrast

In [ ]:
plt.figure(figsize=(10,8))
plt.imshow(((mat_HI)))
plt.title(r"PASTIS matrix $M$", fontsize=20)
plt.xlabel("Mode Index",fontsize=20)
plt.ylabel("Mode Index",fontsize=20)
plt.tick_params(labelsize=15)
cbar = plt.colorbar()
#cbar = plt.colorbar(ticks = np.linspace(-13,-5,8,endpoint=False))
# cbar.ax.set_yticklabels([r'$10^{-13}$', r'$10^{-12}$', r'$10^{-11}$', r'$10^{-10}$', 
#                          r'$10^{-9}$', r'$10^{-8}$', r'$10^{-7}$',r'$10^{-6}$'], fontsize=15)
cbar.set_label(r"in units of $1/{nm^2}$",fontsize =15)
plt.tight_layout()
plt.savefig("/Users/asahoo/Documents/ultra/hi_spatial/mat_HI_100.png")

In [ ]:
filename_matrix1 = 'PASTISmatrix_HI_' + str(n_HI)
hcipy.write_fits(mat_HI, os.path.join(resDir, filename_matrix1 + '.fits'))
print('Matrix saved to:', os.path.join(resDir, filename_matrix1 + '.fits','\n'))

filename_matrix2 = 'EFIELD_Re_matrix_HI_' + str(n_HI)
hcipy.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix2 + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix2 + '.fits', '\n'))

filename_matrix3 = 'EFIELD_Im_matrix_HI_' + str(n_HI)
hcipy.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix3 + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix3 + '.fits','\n'))

### compute tolerance per modal basis

In [ ]:
evals, evecs = np.linalg.eig(mat_HI)
sorted_evals = np.sort(evals)
sorted_indices = np.argsort(evals)
sorted_evecs = evecs[:, sorted_indices]

### plot eigen vectors

In [ ]:
emodes_HI = []
eunit = 1e-9
for mode in range(len(evals)):
    print('Working on mode {}/{}.'.format(mode+1, len(evals)))
    HI_coeffs = np.zeros(n_HI)
    HI_coeffs = eunit*sorted_evecs[:, mode]/2
    luvoir.ripple_mirror.actuators = HI_coeffs
    wf_ripples = luvoir.ripple_mirror(luvoir.wf_aper)
    emodes_HI.append(wf_ripples.phase)

In [ ]:
plt.figure(figsize=(20,20))
for i in range(1, 17):
    plt.subplot(4, 4, i)
    hcipy.imshow_field(emodes_HI[i-1])
    plt.colorbar()

### define your target contrast

In [ ]:
c_target_log = -11
c_target = 10**(c_target_log)
n_repeat = 20

### compute tolerances per sine wave 

In [ ]:
mu_map_HI = np.sqrt(((c_target) / (n_HI)) / (np.diag(mat_HI)))
np.savetxt('/Users/asahoo/Documents/ultra/hi_spatial/mu_map_HI_100_1e-11.csv', mu_map_HI, delimiter=',')

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Modal constraints to achieve a dark hole contrast of "r"$10^{-11}$", fontsize=20)
plt.plot(mu_map_HI)
plt.ylabel("Weight per mode (in units of nm)",fontsize =20)
plt.xlabel("Mode index", fontsize=20)
plt.tick_params(top=False, bottom=True, left=True, right=False,labelleft=True, labelbottom=True, labelsize=20)
plt.tick_params(axis='both',which='major',length=10, width=2)
#plt.tick_params(axis='both',direction='in')
plt.savefig('/Users/asahoo/Documents/ultra/hi_spatial/mu_map_HI_100_1e-11.png')

### Compute tolerance maps for sine wave basis

In [ ]:
nu_maps_HI = []
for qq in range(n_HI):
    HI_coeffs = np.zeros(n_HI)
    HI_coeffs[qq] = mu_map_HI[qq]
    luvoir.ripple_mirror.actuators = HI_coeffs*nm_aber/2
    map_ripple = luvoir.ripple_mirror.surface
    nu_maps_HI.append(map_ripple)

In [ ]:
hcipy.imshow_field(nu_maps_HI[96],mask=luvoir.aperture)
plt.colorbar()

### resizing mu_map

In [ ]:
# mu_map_HI_resized = mu_map_HI[4:100]
# print(len(mu_map_HI_resized))
# plt.plot(mu_map_HI_resized)

## Temporal Ananlysis

### Define stellar parameters

In [ ]:
z_pup_downsample = CONFIG_PASTIS.getfloat('numerical', 'z_pup_downsample') 
N_pup_z = int(luvoir.pupil_grid.shape[0] / z_pup_downsample) #N_pup_z = 100,used to define out-of-band efield
grid_zernike = hcipy.field.make_pupil_grid(N_pup_z, diameter=luvoir.diam)

npup = int(np.sqrt(luvoir.pupil_grid.x.shape[0]))
nimg = int(np.sqrt(luvoir.focal_det.x.shape[0]))

# Getting the flux together
sptype = 'A0V'
Vmag = 5.0
minlam = 500
maxlam = 600 
dark_current = 0     
CIC = 0            
star_flux = exoscene.star.bpgs_spectype_to_photonrate(spectype=sptype, Vmag=Vmag, minlam=minlam, maxlam=maxlam) #ph/s/m^2
Nph = star_flux.value*15**2*np.sum(luvoir.apodizer**2) / npup**2

### compute out of band reference electric field seen by the wfs (here it is a zernike wfs)

In [ ]:
luvoir.ripple_mirror.flatten()
ob_ref2 = luvoir.calc_out_of_band_wfs(norm_one_photon=True) #returns wavefront on obwfs detector
ob_ref2_sub_real = hcipy.field.subsample_field(ob_ref2.real, z_pup_downsample, grid_zernike, statistic='mean')
ob_ref2_sub_imag = hcipy.field.subsample_field(ob_ref2.imag, z_pup_downsample, grid_zernike, statistic='mean')
Efield_ref_OBWFS = (ob_ref2_sub_real + 1j*ob_ref2_sub_imag) * z_pup_downsample

In [ ]:
plt.figure(figsize=(30,10))

plt.subplot(2,4,1)
plt.title("amplitude")
hcipy.imshow_field(ob_ref2.amplitude)
plt.colorbar()

plt.subplot(2,4,2)
plt.title("phase")
hcipy.imshow_field(ob_ref2.phase)
plt.colorbar()

plt.subplot(2,4,3)
plt.title("real")
hcipy.imshow_field(ob_ref2.real)
plt.colorbar()

plt.subplot(2,4,4)
plt.title("imag")
hcipy.imshow_field(ob_ref2.imag)
plt.colorbar()

plt.subplot(2,4,5)
plt.title("real_sub")
hcipy.imshow_field(ob_ref2.real)
plt.colorbar()

plt.subplot(2,4,6)
plt.title("imag_sub")
hcipy.imshow_field(ob_ref2.imag)
plt.colorbar()

plt.subplot(2,4,7)
plt.title("Efield_ref_OBWFS.real")
hcipy.imshow_field(Efield_ref_OBWFS.real)
plt.colorbar()

plt.subplot(2,4,8)
plt.title("Efield_ref_OBWFS.imag")
hcipy.imshow_field(Efield_ref_OBWFS.imag)
plt.colorbar()
plt.show()

### reducing to nyquist sampling

In [ ]:
nyquist_sampling = 2.

# Actual grid for LUVOIR images
grid_test = hcipy.make_focal_grid(
            luvoir.sampling,
            luvoir.imlamD,
            pupil_diameter=luvoir.diam,
            focal_length=1,
            reference_wavelength=luvoir.wvln,
        )

# Actual grid for LUVOIR images that are nyquist sampled
grid_det_subsample = hcipy.make_focal_grid(
            nyquist_sampling,
            np.floor(luvoir.imlamD),
            pupil_diameter=luvoir.diam,
            focal_length=1,
            reference_wavelength=luvoir.wvln,
        )
n_nyquist = int(np.sqrt(grid_det_subsample.x.shape[0]))

In [ ]:
design = 'small'

dh_outer_nyquist = hcipy.circular_aperture(2 * luvoir.apod_dict[design]['owa'] * luvoir.lam_over_d)(grid_det_subsample)
dh_inner_nyquist = hcipy.circular_aperture(2 * luvoir.apod_dict[design]['iwa'] * luvoir.lam_over_d)(grid_det_subsample)
dh_mask_nyquist = (dh_outer_nyquist - dh_inner_nyquist).astype('bool')

dh_size = len(np.where(luvoir.dh_mask != 0)[0])
dh_size_nyquist = len(np.where(dh_mask_nyquist != 0)[0])
dh_index = np.where(luvoir.dh_mask != 0)[0]
dh_index_nyquist = np.where(dh_mask_nyquist != 0)[0]

### define out of band electric field seen by wfs

In [ ]:
E0_OBWFS = np.zeros([N_pup_z*N_pup_z,1,2])
E0_OBWFS[:,0,0] = Efield_ref_OBWFS.real
E0_OBWFS[:,0,1] = Efield_ref_OBWFS.imag

### coronagraphic efield

In [ ]:
E0_coron = np.zeros([nimg*nimg,1,2])
E0_coron[:,0,0] = Efield_ref.real 
E0_coron[:,0,1] = Efield_ref.imag

In [ ]:
plt.figure(figsize=(10,10))

plt.subplot(2,2,1)
plt.title("Efield_ref.real")
plt.imshow(np.array(np.reshape((E0_OBWFS[:,0,0]),(100,100))))
plt.colorbar()

plt.subplot(2,2,2)
plt.title("Efield_ref.imag")
plt.imshow(np.array(np.reshape((E0_OBWFS[:,0,1]),(100,100))))
plt.colorbar()

plt.subplot(2,2,3)
plt.title("coron_Efield_ref.real")
plt.imshow(np.array(np.reshape((E0_coron[:,0,0]),(115,115))))
plt.colorbar()

plt.subplot(2,2,4)
plt.title("coron_Efield_ref.imag")
plt.imshow(np.array(np.reshape((E0_coron[:,0,1]),(115,115))))
plt.colorbar()

### sensitivity matrix at the coronagraph plane, this can be saved during the initial pastis matrix generation

In [ ]:
filename_matrix2 = 'EFIELD_Re_matrix_HI_' + str(n_HI) + '.fits'
G_HI_real = fits.getdata(os.path.join(overall_dir, 'matrix_numerical', filename_matrix2))
filename_matrix3 = 'EFIELD_Im_matrix_HI_' + str(n_HI) + '.fits'
G_HI_imag = fits.getdata(os.path.join(overall_dir, 'matrix_numerical', filename_matrix3))

In [ ]:
G_coron_HI= np.zeros([nimg*nimg,2,n_HI])
for pp in range(0, n_HI):
    G_coron_HI[:,0,pp] = G_HI_real[pp] - Efield_ref.real
    G_coron_HI[:,1,pp] = G_HI_imag[pp] - Efield_ref.imag

### sensitivity matrix for out-of-band wfs

In [ ]:
start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

for i in range(0, n_HI):
    #print(f'Working on "defocus" zernike mode, segment: {i}')
    
    # Apply calibration aberration to used mode
    HI_modes = np.zeros(len(mu_map_HI))
    HI_modes[i] = (nm_aber)/2
    luvoir.ripple_mirror.actuators  = HI_modes
    HI_meas = luvoir.calc_out_of_band_wfs(norm_one_photon=True)
    HI_meas_sub_real = hcipy.field.subsample_field(HI_meas.real, z_pup_downsample, grid_zernike, statistic='mean')
    HI_meas_sub_imag = hcipy.field.subsample_field(HI_meas.imag, z_pup_downsample, grid_zernike, statistic='mean')
    focus_field1 = HI_meas_sub_real + 1j * HI_meas_sub_imag
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

In [ ]:
filename_matrix = 'EFIELD_OBWFS_Re_HI_' + str(n_HI)
hcipy.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix + '.fits'))

filename_matrix = 'EFIELD_OBWFS_Im_HI_' + str(n_HI)
hcipy.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix + '.fits'))

In [ ]:
filename_matrix = 'EFIELD_OBWFS_Re_HI_' + str(n_HI)+'.fits'
G_OBWFS_real = fits.getdata(os.path.join(overall_dir, 'matrix_numerical', filename_matrix))
filename_matrix = 'EFIELD_OBWFS_Im_HI_' + str(n_HI)+'.fits'
G_OBWFS_imag =  fits.getdata(os.path.join(overall_dir, 'matrix_numerical', filename_matrix))

In [ ]:
G_OBWFS= np.zeros([N_pup_z*N_pup_z,2,n_HI])
for pp in range(0, n_HI-1):
    G_OBWFS[:,0,pp] = G_OBWFS_real[pp]*z_pup_downsample - Efield_ref_OBWFS.real
    G_OBWFS[:,1,pp] = G_OBWFS_imag[pp]*z_pup_downsample - Efield_ref_OBWFS.imag

### close loop batch estimation algorithm

In [ ]:
flux = Nph
Qhi = np.diag(np.asarray(mu_map_HI**2))

Ntimes = 20
TimeMinus = -2
TimePlus = 5.5 #3.5
Nwavescale = 8
Nflux = 3

res = np.zeros([Ntimes, Nwavescale, Nflux, 1])
result_wf_test =[]

#i=-1
for wavescale in range (1,15,2):
    #i=i+1
    print('Harris modes with batch OBWFS and noise %f'% wavescale, "i",i)  
    niter = 10
    timer1 = time.time()
    StarMag = 0.0
    #j=-1
    for tscale in np.logspace(TimeMinus, TimePlus, Ntimes):
        j=j+1
        Starfactor = 10**(-StarMag/2.5)
        print(tscale)
        tmp0 = req_closedloop_calc_batch(G_coron_HI, G_OBWFS, E0_coron, E0_OBWFS, dark_current+CIC/tscale,
                                                 dark_current+CIC/tscale, tscale, flux*Starfactor, 0.0001*wavescale**2*Qhi,
                                                 niter, luvoir.dh_mask, norm)    
        tmp1 = tmp0['averaged_hist']
        n_tmp1 = len(tmp1)
        result_wf_test.append(tmp1[n_tmp1-1])

In [ ]:
delta_wf = []
for wavescale in range (1,15,2):
    wf = 1e3*np.sqrt(0.0001*wavescale**2)
    delta_wf.append(wf)

texp = np.logspace(TimeMinus, TimePlus, Ntimes)

font = {'family': 'serif','color' : 'black','weight': 'normal','size'  :  20}
plt.figure(figsize =(15,10))

plt.title('Target contrast = %s, Vmag= %s'%(c_target, Vmag),fontdict=font)
plt.plot(texp,result_wf_test[0:20]-contrast_floor, label=r'$\Delta_{wf}= %d\ pm$'%(delta_wf[0]))
plt.plot(texp,result_wf_test[20:40]-contrast_floor, label=r'$\Delta_{wf}=%d\ pm$'%(delta_wf[1]))
plt.plot(texp,result_wf_test[40:60]-contrast_floor, label=r'$\Delta_{wf}=%d\ pm$'%(delta_wf[2]))
plt.plot(texp,result_wf_test[60:80]-contrast_floor, label=r'$\Delta_{wf}=%d\ pm$'%(delta_wf[3]))
plt.plot(texp,result_wf_test[80:100]-contrast_floor, label=r'$\Delta_{wf}=%d\ pm$'%(delta_wf[4]))
plt.plot(texp,result_wf_test[100:120]-contrast_floor, label=r'$\Delta_{wf}=%d\ pm$'%(delta_wf[5]))
plt.plot(texp,result_wf_test[120:140]-contrast_floor, label=r'$\Delta_{wf}=%d\ pm$'%(delta_wf[6]))
plt.xlabel("$t_{WFS}$ in secs",fontsize=20)
plt.ylabel("$\Delta$ contrast",fontsize=20)
plt.yscale('log')
plt.xscale('log')
plt.legend(loc = 'upper center',fontsize=20)
plt.tick_params(top=False, bottom=True, left=True, 
                right=True,labelleft=True, labelbottom=True,
                labelsize=20)
plt.tick_params(axis='both',which='major',length=10, width=2)
plt.tick_params(axis='both',which='minor',length=6, width=2)
plt.grid()
plt.savefig('/Users/asahoo/Documents/ultra/hi_spatial/c_texp_1e-11.png')
plt.show()

### some junk random codes

In [ ]:
emodes_HI = []
eunit = 1e-9
for mode in range(len(evals)):
    print('Working on mode {}/{}.'.format(mode+1, len(evals)))
    HI_coeffs = np.zeros(n_HI)
    HI_coeffs = eunit*sorted_evecs[:, mode]/2
    luvoir.ripple_mirror.actuators = HI_coeffs
    wf_ripples = luvoir.ripple_mirror(luvoir.wf_aper)
#     emodes_HI.append(wf_ripples.phase)